# Science Demonstration Case: Polar Science
# Automatic ice damage detection from Sentinel-1 radar imagery
## Step 2: Get Sentinel-1 imagery data.
### For more info about this science case, please check the documentation [at this link](https://earthsystemdatalab.net/science_cases/polar_science/).

In this notebook SentinelHub is exploited to query, preprocess and download Sentinel-1 data.<br>
The imagery data is then saved into a datacube, with the same resolution and extent as the damage model.

NB: A SentinelHub account is required in order to use its functionalities. By default the `sh.SHConfig()` instance will look for the `SH_CLIENT_ID` and `SH_CLIENT_SECRET` environment variables.<br>
If they are not set, they can be passed to the `sh.SHConfig()` instance after its creation by running `sh_config.sh_client_id = <ID>` and `sh_config.sh_client_secret = <SECRET>`.

**This notebook runs with the python environment `users-deepesdl-deep-code`, please checkout the documentation for [help on changing the environment](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kerne).**

In [73]:
# import the necessary packages

import os
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import xarray as xr
import rioxarray
from xcube.core.store import new_data_store
import sentinelhub as sh

In [81]:
# define the parameters needed to access the S3 storage
# NB: these are personal details, hence during the development of these notebooks they have been saved as environment variables;
# remember to change them depending on the user's necessity

S3_USER_STORAGE_KEY = os.environ['S3_USER_STORAGE_KEY']
S3_USER_STORAGE_SECRET = os.environ['S3_USER_STORAGE_SECRET']
S3_USER_STORAGE_BUCKET = os.environ['S3_USER_STORAGE_BUCKET']

team_store = new_data_store(
    data_store_id='s3',
    root=S3_USER_STORAGE_BUCKET,
    storage_options=dict(
        anon=False, key=S3_USER_STORAGE_KEY,
        secret=S3_USER_STORAGE_SECRET
    )
)

In [ ]:
INPUT_DATA_DIRECTORY = Path.cwd() / 'sentinelhub_datafolder'

# list of the available ice shelves

ICESHELF_LIST = ['amery', 'amundsen', 'getz']

In [79]:
# the example shown is for the Amery Ice Shelf

iceshelf_name = 'amery'
iceshelf_directory = INPUT_DATA_DIRECTORY / iceshelf_name

<b>Coordinate Reference Systems</b>

- EPSG 3031 is Antarctic Polar Stereographic projected coordinates
- UTM depends on the longitude of the area of interest and is needed to specify the resolution / size of the imagery data, due to how SentinelHub is implemented



<b>Bounding box of the area of interest</b>

In [ ]:
amery_iceshelf_bbox = sh.BBox((1566250, 533250, 2266750, 933750), crs=sh.CRS(3031))

# resolution in [m], in EPSG:3031 projected coordinates
resolution_x, resolution_y = (500, 500)
# size of the imagery array given the resolution specified above
imagery_width = round(abs(amery_iceshelf_bbox.max_x - amery_iceshelf_bbox.min_x) / resolution_x)
imagery_height = round(abs(amery_iceshelf_bbox.max_y - amery_iceshelf_bbox.min_y) / resolution_y)

<b>Time interval of interest</b>

As an example, let's download the data for the last quarter of 2016 (Q4). The science case actually employed data from both Q4 of 2015 and 2016.

In [57]:
time_start = datetime(2016, 10, 1)
time_end = datetime(2016, 12, 31)
# set the time interval in the format required by the SentinelHub API
time_interval = time_start.strftime('%Y-%m-%d'), time_end.strftime('%Y-%m-%d')

# this is useful to distinguish the files in case data is downloaded for multiple time intervals
time_period = '2016_Q4'

<b>Sentinel-1 categories</b>
- Acquisition mode
    - IW: Interferometric Wide swath mode
    - EW: Extra Wide swath mode
- Polarisation
    - SH: only HH co-polarisation
    - SV: only VV co-polarisation
    - DH: double polarisation, both co- (HH) and cross- (HV)
    - DV: double polarisation, both co- (VV) and cross- (VH)
- Orbit
    - Ascending
    - Descending

<b>Evalscripts custom scripts</b>

These are the v3 evalscripts useful for the polar regions, where DH and SH polarisation modes are available.
See the SentinelHub documentation at [this link](https://docs.sentinel-hub.com/api/latest/evalscript/v3/) for an explanation of what the evalscripts are and what they are used for.
<br>
The latter is used in this notebook, to get all the tiles in single polarisation mode containing only co-polarised data.

In [58]:
evalscript_s1_pol_dh = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["HH", "HV"]
            }],
            output: {
                bands: 2, sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.HH, sample.HV];
    }
"""

evalscript_s1_pol_sh = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["HH"]
            }],
            output: {
                bands: 1, sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.HH];
    }
"""

<b>Find the Sentinel-1 tiles that overlap the area of interest</b>

In [59]:
# set up the Sentinel Hub Config object

sh_config = sh.SHConfig()
sh_config.sh_base_url = sh.DataCollection.SENTINEL1.service_url
sh_catalog = sh.SentinelHubCatalog(config=sh_config)

# the search is performed only based on the bounding box and the time interval
search_iterator = sh_catalog.search(
    sh.DataCollection.SENTINEL1,
    bbox=amery_iceshelf_bbox,
    time=time_interval,
    fields={'include': ['id', 'properties.datetime'],
            'exclude': []},
)

results = list(search_iterator)
print('Total number of results:', len(results))

Total number of results: 229


<b>Preprocess and download the tiles and save them as data cubes</b>

Together with applying the custom evalscript, the desired preprocessing steps are also defined.

In [70]:
# define a (narrow) time window (in this case of 1 minute) within which subsequent scenes can be joined into the same mosaic
time_difference = timedelta(seconds=60)

for sentinel_tile_id, sentinel_tile_timestamp in zip(
    search_iterator.get_ids()[1:],
    search_iterator.get_timestamps()[1:]):
    polarization = sh_catalog.get_feature(sh.DataCollection.SENTINEL1, sentinel_tile_id)['properties']['s1:polarization']
    orbit = sh_catalog.get_feature(sh.DataCollection.SENTINEL1, sentinel_tile_id)['properties']['sat:orbit_state']
    tile_data_folder = iceshelf_directory / orbit / f'{polarization.lower()}_data' / time_period / f"{sentinel_tile_timestamp.strftime('%Y%m%dT%H%M%S')}"
    zarr_output_file_path = iceshelf_directory / orbit / f'{polarization.lower()}_data' / time_period / f'{sentinel_tile_id}.zarr'

    if polarization == 'SH':
        request = sh.SentinelHubRequest(
            evalscript=evalscript_s1_pol_sh,
            input_data=[
                sh.SentinelHubRequest.input_data(
                    data_collection=sh.DataCollection.SENTINEL1,
                    time_interval=(
                        sentinel_tile_timestamp - time_difference,
                        sentinel_tile_timestamp + time_difference),
                    other_args={'processing': {
                        'orthorectify': True,
                        'demInstance': 'COPERNICUS',
                        'backCoeff': 'GAMMA0_TERRAIN'}}
                )
            ],
            responses=[sh.SentinelHubRequest.output_response('default', MimeType.TIFF)],
            bbox=amery_iceshelf_bbox,
            size=(imagery_width, imagery_height),
            config=sh_config,
            data_folder=tile_data_folder.as_posix(),
        )

    # get only SH data, so if a file is DH don't make any request
    elif polarization == 'DH':
        continue

    downloaded_data = sh.SentinelHubDownloadClient(config=sh_config).download(request.download_list[0], max_threads=1)
    tiff_path = Path(request.download_list[0].data_folder) / request.get_filename_list()[0]
    tiff_file = xr.open_mfdataset(tiff_path, engine='rasterio')
    imagery_datacube = tiff_file.isel(band=0).drop_vars('band').expand_dims(
        dim={'time': [int(sentinel_tile_timestamp.timestamp())]},
        axis=0, create_index_for_new_dim=True)
    imagery_datacube = imagery_datacube.reindex(y = imagery_datacube.y[::-1])
    imagery_datacube = imagery_datacube.rename({'band_data': 's1_imagery'})
    imagery_datacube = imagery_datacube.chunk(dict(time=1, x=128, y=128))
    imagery_datacube.to_zarr(zarr_output_file_path)

/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/sentinelhub/download/client.py:79: SHDeprecationWarning: The parameter `download_requests` should be a sequence of requests. In future versions download of single requests will only be supported if provided as a singelton tuple or list.
  warnings.warn(
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/sentinelhub/download/client.py:79: SHDeprecationWarning: The parameter `download_requests` should be a sequence of requests. In future versions download of single requests will only be supported if provided as a singelton tuple or list.
  warnings.warn(
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/sentinelhub/download/client.py:79

<b>Combine the data cubes and save them to S3 storage</b>

In [84]:
orbit_type = 'ascending'
polarization_mode = 'SH'
polarization_type = 'HH'
time_period = '2016_Q4'
data_folder = iceshelf_directory / orbit_type / f'{polarization_mode.lower()}_data' / time_period
tiles_list = list(data_folder.glob('*.zarr'))
tiles_list.sort(key=os.path.getmtime, reverse=True)
imagery_datacube = xr.open_mfdataset(
    tiles_list, drop_variables=['spatial_ref'])

del imagery_datacube.s1_imagery.attrs['AREA_OR_POINT']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_RESOLUTIONUNIT']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_XRESOLUTION']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_YRESOLUTION']
del imagery_datacube.s1_imagery.attrs['grid_mapping']
imagery_datacube['s1_imagery'].attrs = {
    'description': 'Imagery tiles',
    'units': 'Normalised backscatter (linear power).'}
imagery_datacube = imagery_datacube.rio.write_crs('epsg:3031', grid_mapping_name='crs')
imagery_datacube = imagery_datacube.assign_attrs(
    description=f"Sentinel-1 imagery over Amery Ice Shelf for the period {time_period.replace('_', ' ')}. \
Polarization mode: {polarization_mode}. Polarization: {polarization_type}. Orbit: {orbit_type}.")

storage_path = f'datacubes/{iceshelf_name}/s1_imagery_res_500m_{polarization_mode}_{polarization_type}_{orbit_type.capitalize()}_{time_period}.zarr'
team_store.write_data(
    imagery_datacube, storage_path, replace=False)

/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


'datacubes/amery/s1_imagery_res_500m_SH_HH_Ascending_2016_Q4.zarr'

In [85]:
orbit_type = 'descending'
polarization_mode = 'SH'
polarization_type = 'HH'
time_period = '2016_Q4'
data_folder = iceshelf_directory / orbit_type / f'{polarization_mode.lower()}_data' / time_period
tiles_list = list(data_folder.glob('*.zarr'))
tiles_list.sort(key=os.path.getmtime, reverse=True)
imagery_datacube = xr.open_mfdataset(
    tiles_list, drop_variables=['spatial_ref'])

del imagery_datacube.s1_imagery.attrs['AREA_OR_POINT']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_RESOLUTIONUNIT']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_XRESOLUTION']
del imagery_datacube.s1_imagery.attrs['TIFFTAG_YRESOLUTION']
del imagery_datacube.s1_imagery.attrs['grid_mapping']
imagery_datacube['s1_imagery'].attrs = {
    'description': 'Imagery tiles',
    'units': 'Normalised backscatter (linear power).'}
imagery_datacube = imagery_datacube.rio.write_crs('epsg:3031', grid_mapping_name='crs')
imagery_datacube = imagery_datacube.assign_attrs(
    description=f"Sentinel-1 imagery over Amery Ice Shelf for the period {time_period.replace('_', ' ')}. \
Polarization mode: {polarization_mode}. Polarization: {polarization_type}. Orbit: {orbit_type}.")

storage_path = f'datacubes/{iceshelf_name}/s1_imagery_res_500m_{polarization_mode}_{polarization_type}_{orbit_type.capitalize()}_{time_period}.zarr'
team_store.write_data(
    imagery_datacube, storage_path, replace=False)

/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/conda/deepesdl/b1713238edc260f848ded763d46a765b1625abc7984a5240701aaa391923b034-20240516-131736-043804-461-ml-2024.05/lib/python3.10/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


'datacubes/amery/s1_imagery_res_500m_SH_HH_Descending_2016_Q4.zarr'